# 0. Importing packages

In [ ]:
from scipy.optimize import differential_evolution
import numpy as np
from matplotlib import pyplot as plt

from packages.functions import calcL1MET
import optimisation_2024 as opt

# 1. Loading data

In [ ]:
data2024 = "/shared/scratch/wq22321/data/NANOAOD_Muon0Run2023D_ZMu_PromptReco_v2RAW_RECO_2023_v0_4/231121_100830/0000/out_10.root"

In [ ]:
fit, valid = opt.prepareInputs(dir = data2024, subset=0.7, cuts=(0, 250))

# 2. Optimisation

In [ ]:
print("Starting optimisation")

bounds = [(0, 3), (0, 4), (0, 3), (0, 4)]
x0 = (2.0, 2.0, 0.5, 2.0)
turn_on_options = (True, 80, 0.05)
result = differential_evolution(
    func     = opt.objective,
    bounds   = bounds,
    args     = (fit, turn_on_options),
    
    x0 = x0,
    popsize  = 15,    # 15
    maxiter  = 10,    # 1000
    strategy = "best1bin",    # "best1bin"
    init     = "sobol",    #"latinhypercube"
    disp     = True,
    workers  = -1,    # 1
    polish   = False
    )

print(result.x)
print(result)

In [ ]:
result.x

# 3. Parameter analysis

In [ ]:
def heatmapper(a, b, c, d):
    dat = lookup_gen(a, b, c, d)
    ietas, pu_bins, thresh = zip(*dat)
    plt.figure(figsize=(15,9))
    plt.hist2d(ietas, pu_bins, weights=thresh, bins=[83, 32])
    plt.colorbar()
    #plt.xlim([1,41]); plt.ylim([0,31])
    plt.xlabel("ieta"); plt.ylabel("NTT4 bin")
    plt.title("2D histogram of tower threshold (scaled by tower size)")
    plt.show()

In [ ]:
def lookup_gen(a, b, c, d):
    
    all_ieta_vals = np.linspace(-41, 41, 83)
    all_pu_bins = np.linspace(0, 31, 32)
    
    res = []
    for ieta in all_ieta_vals:
        for pu_bin in all_pu_bins:
            thresh = threshold_calc(ieta, pu_bin, a, b, c, d)
            res.append((ieta, pu_bin, thresh))
            
    return res

In [ ]:
def threshold_calc(ieta, ntt4, a, b, c, d):
    towerAreas = [    0., # dummy for ieta=0
                  1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,
                  1.,1.,1.,1.,1.,1.,1.,1.,1.,1.,
                  1.03,1.15,1.3,1.48,1.72,2.05,1.72,4.02,
                  0., # dummy for ieta=29
                  3.29,2.01,2.02,2.01,2.02,2.0,2.03,1.99,2.02,2.04,2.00,3.47]
    
    numerator = (towerAreas[int(abs(ieta))]**a) * (ntt4**c)
    denominator = d * (1 + np.exp(-b * (abs(ieta))))
    
    threshold = (numerator / denominator).clip(max=40)
#    return (threshold/2)# / towerAreas[int(abs(ieta))]
    if towerAreas[int(abs(ieta))] == 0:
        return np.nan
    else:
        return (threshold/2)# / towerAreas[int(abs(ieta))]

In [ ]:
fw = (0.66881105, 1.77791976, 0.96226907, 2.08912586)
reg = (0.21456065, 2.74961211, 0.47896413, 0.30650943)
old = (1.707, 3.078, 0.195, 1.365)

quick = (2.49418849, 3.06860578, 0.22999811, 3.75036927)
quick2 = (0.4785183 , 2.95870881, 0.19865517, 1.69915227)

zmu23_1 = (1.87, 1.34, 0.1, 2.84)
zmu23_2 = (1.49, 2.91, 0.26, 1.51)
zmu23_3 = (2.34, 2.82, 0.37, 2.79)

In [ ]:
heatmapper(*quick2)

In [ ]:
valid_calo, valid_puppi, valid_ntt4 = valid

In [ ]:
MET = opt.applyCaloTowerThresh(valid_calo, *quick2)[0]

In [ ]:
l1met = calcL1MET(valid_calo)

In [ ]:
def getTurnOn( online, offline, threshold=80 ) :
    offline_bins = np.linspace(0, 300, 40)
    efficiency = []


    for i in range(len(offline_bins) - 1):
        # Define the offline range for this bin
        offline_range = (offline >= offline_bins[i]) & (offline < offline_bins[i + 1])
        # count the number of events passing the threshold in the offline range
        num_offline = sum(offline_range)
        # count the number of events passing the threshold in both online and offline ranges
        num_both = sum((online > threshold) & offline_range)
        # calculate the efficiency as the ratio of online events passing the cut over offline events passing the threshold
        if num_offline > 0:
            eff = num_both / num_offline
        else:
            eff = 0
        efficiency.append(eff)

    bin_centers = (offline_bins[:-1] + offline_bins[1:]) / 2

    return bin_centers, efficiency

In [ ]:
bins, turnon = getTurnOn(MET, valid_puppi)

In [ ]:
_, turnon_noPU = getTurnOn(l1met, valid_puppi)

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(bins, turnon, label="With PU sup")
plt.plot(_, turnon_noPU, label = "No PU sup")

plt.grid(True, linestyle='--', color='gray', alpha=0.5)
plt.hlines(0.95, xmin=0, xmax=300, colors='gray', linestyles='--', alpha=0.5)
plt.legend()
plt.show()